## Library Import

In [1]:
import pandas as pd
from matplotlib import pyplot
from pandas.plotting import lag_plot
from IPython.display import display,Markdown
import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import statistics as stats
import warnings
warnings.simplefilter('ignore')
import datetime as dt

'''
import fbprophet
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.plot import plot_forecast_component
from prophet_plots_plotly import plot_model_components, plot_model
'''
buttons_to_remove = ['zoom2d','zoomIn2d','zoomOut2d',
                     'sendDataToCloud',
                     'editInChartStudio',
                     'select2d',
                     'lasso2d',]

scatter_config = {'displayModeBar': 'Always',
                 "displaylogo": False,
                 'modeBarButtons': 'toggleHover ',
                 'modeBarButtonsToRemove': buttons_to_remove,}

legend_config = dict(orientation = 'h',
                 yanchor="bottom",
                 y=1.05,
                 xanchor="left",
                 x=0.01)


## Import & Modify CPI Data

In [2]:
'''
Raw CPI Data
'''

cpi_raw = pd.read_excel('AllItems_CPIRawData.xlsx',engine='openpyxl',sheet_name = 'BLS Data Series',skiprows=11)
cpi_raw = cpi_raw.loc[:,~cpi_raw.columns.str.startswith('HALF')]

cpi_raw.set_index('Year',inplace = True)
cpi_raw['Annual'] = cpi_raw.mean(numeric_only=True, axis=1)
cpi_raw.reset_index(inplace = True)
display(Markdown('<h2>Raw CPI Data</h2>'))
display(cpi_raw.head())

'''
Import CPI data, remove monthly data, calculate inflation, and find rolling avg of CPI and inflation
'''

cpi_yearly = cpi_raw.copy()

cpi_yearly = cpi_yearly[['Year','Annual']]
cpi_yearly.rename(columns={"Annual": "CPI",'Year': 'Date'},inplace=True)

window_yearly = 3

cpi_yearly['RollingAvgCPI'] = cpi_yearly.CPI.rolling(window_yearly).mean()
cpi_yearly['RollingAvgCPI_low'] = cpi_yearly.CPI.rolling(window_yearly).min()
cpi_yearly['RollingAvgCPI_high'] = cpi_yearly.CPI.rolling(window_yearly).max()

cpi_yearly['Inflation'] = (cpi_yearly.CPI.diff()/ cpi_yearly['CPI'].shift(1))*100
cpi_yearly['RollingAvgInflation'] = cpi_yearly.Inflation.rolling(window_yearly).mean()
cpi_yearly['RollingAvgInflation_low'] = cpi_yearly.Inflation.rolling(window_yearly).min()
cpi_yearly['RollingAvgInflation_high'] = cpi_yearly.Inflation.rolling(window_yearly).max()

display(Markdown('<h2>Yearly CPI and Inflation Data</h2>'))
display(cpi_yearly.head())


'''
Transform CPI data into monthly format and calculate inflation
'''
cpi_monthly =  cpi_raw.copy()
#drop unnecessary columns
cpi_monthly.drop(columns = ['Annual'], inplace = True)

window_monthly = 6

#use pd.melt to combine columns and rows
cpi_monthly = pd.melt(cpi_monthly, id_vars=["Year"], var_name="Month", value_name = "CPI")
#modify columns to get usable format and calculate inflation
cpi_monthly['Date'] = cpi_monthly.Month + " " + cpi_monthly.Year.map(str)
cpi_monthly = cpi_monthly[['Date','CPI']]
cpi_monthly["Date"] = pd.to_datetime(cpi_monthly.Date, format="%b %Y", dayfirst=True)
cpi_monthly = cpi_monthly.sort_values("Date")
#cpi_monthly ['Date'] = cpi_monthly['Date'].dt.strftime('%m/%d/%Y')
cpi_monthly ['Date'] = cpi_monthly['Date'].dt.date
cpi_monthly['Inflation'] = (cpi_monthly.CPI.diff()/ cpi_monthly['CPI'].shift(1))*100

cpi_monthly['RollingAvgCPI'] = cpi_monthly.CPI.rolling(window_monthly).mean()
cpi_monthly['RollingAvgCPI_low'] = cpi_monthly.CPI.rolling(window_monthly).min()
cpi_monthly['RollingAvgCPI_high'] = cpi_monthly.CPI.rolling(window_monthly).max()

cpi_monthly['Inflation'] = (cpi_monthly.CPI.diff()/ cpi_monthly['CPI'].shift(1))*100
cpi_monthly['RollingAvgInflation'] = cpi_monthly.Inflation.rolling(window_monthly).mean()
cpi_monthly['RollingAvgInflation_low'] = cpi_monthly.Inflation.rolling(window_monthly).min()
cpi_monthly['RollingAvgInflation_high'] = cpi_monthly.Inflation.rolling(window_monthly).max()

display(Markdown('<h2>Monthly CPI and Inflation Data</h2>'))
display(cpi_monthly.head())

<h2>Raw CPI Data</h2>

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,1947,21.48,21.62,22.00,22.00,21.95,22.08,22.23,22.40,22.84,22.91,23.06,23.41,22.331667
1,1948,23.68,23.67,23.50,23.82,24.01,24.15,24.40,24.43,24.36,24.31,24.16,24.05,24.045000
2,1949,24.01,23.91,23.91,23.92,23.91,23.92,23.70,23.70,23.75,23.67,23.70,23.61,23.809167
3,1950,23.51,23.61,23.64,23.65,23.77,23.88,24.07,24.20,24.34,24.50,24.60,24.98,24.062500
4,1951,25.38,25.83,25.88,25.92,25.99,25.93,25.91,25.86,26.03,26.16,26.32,26.47,25.973333


<h2>Yearly CPI and Inflation Data</h2>

,Date,CPI,RollingAvgCPI,RollingAvgCPI_low,RollingAvgCPI_high,Inflation,RollingAvgInflation,RollingAvgInflation_low,RollingAvgInflation_high
0,1947,22.331667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1948,24.045000,NaN,NaN,NaN,7.672214,NaN,NaN,NaN
2,1949,23.809167,23.395278,22.331667,24.045000,-0.980800,NaN,NaN,NaN
3,1950,24.062500,23.972222,23.809167,24.062500,1.064016,2.585143,-0.9808,7.672214
4,1951,25.973333,24.615000,23.809167,25.973333,7.941126,2.674781,-0.9808,7.941126


<h2>Monthly CPI and Inflation Data</h2>

,Date,CPI,Inflation,RollingAvgCPI,RollingAvgCPI_low,RollingAvgCPI_high,RollingAvgInflation,RollingAvgInflation_low,RollingAvgInflation_high
0,1947-01-01,21.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,1947-02-01,21.62,0.651769,NaN,NaN,NaN,NaN,NaN,NaN
152,1947-03-01,22.00,1.757632,NaN,NaN,NaN,NaN,NaN,NaN
228,1947-04-01,22.00,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
304,1947-05-01,21.95,-0.227273,NaN,NaN,NaN,NaN,NaN,NaN


## Plot CPI and inflation data with rolling avg

In [3]:
'''
CPI Yearly
'''

fig = go.Figure()

y = cpi_yearly['CPI']
x = cpi_yearly['Date']
y_roll = cpi_yearly['RollingAvgCPI']
y_rollmin = cpi_yearly['RollingAvgCPI_low']
y_rollmax = cpi_yearly['RollingAvgCPI_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Yearly CPI'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window_yearly} Year Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window_yearly} Year Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window_yearly} Year Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
fig.update_layout(legend=legend_config)

display(Markdown('<h2>Yearly CPI Data</h2>'))
fig.show(config=scatter_config)

'''
CPI Monthly
'''

fig = go.Figure()

y = cpi_monthly['CPI']
x = cpi_monthly['Date']
y_roll = cpi_monthly['RollingAvgCPI']
y_rollmin = cpi_monthly['RollingAvgCPI_low']
y_rollmax = cpi_monthly['RollingAvgCPI_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Monthly CPI'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window_monthly} Month Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window_monthly} Month Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window_monthly} Month Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
fig.update_layout(legend=legend_config)

display(Markdown('<h2>Monthly CPI Data</h2>'))
fig.show(config=scatter_config)

'''
Inflation Yearly
'''

fig = go.Figure()

y = cpi_yearly['Inflation']
x = cpi_yearly['Date']
y_roll = cpi_yearly['RollingAvgInflation']
y_rollmin = cpi_yearly['RollingAvgInflation_low']
y_rollmax = cpi_yearly['RollingAvgInflation_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Yearly Inflation'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window_yearly} Year Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window_yearly} Year Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window_yearly} Year Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
fig.update_layout(legend=legend_config)

display(Markdown('<h2>Yearly Inflation Data</h2>'))
fig.show(config=scatter_config)


'''
Inflation Monthly
'''

fig = go.Figure()

y = cpi_monthly['Inflation']
x = cpi_monthly['Date']
y_roll = cpi_monthly['RollingAvgInflation']
y_rollmin = cpi_monthly['RollingAvgInflation_low']
y_rollmax = cpi_monthly['RollingAvgInflation_high']

fig.add_trace(go.Scatter(x=x, y=y,
                         marker=dict(color='#005100',size = 5),
                         line = dict(color='#005100',width = 1),
                         mode='markers+lines',
                         name='Monthly Inflation'))

fig.add_trace(go.Scatter(x=x, y=y_roll,
                         line = dict(color='#FF3333',width = 1),
                         mode='lines',
                         name=f'{window_monthly} Month Rolling Avg'))

fig.add_trace(go.Scatter(x=x, y=y_rollmin,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         name=f'{window_monthly} Month Rolling Low'))

fig.add_trace(go.Scatter(x=x, y=y_rollmax,
                         line = dict(color='#bf9fa2',width = 1),
                         mode='lines',
                         fill='tonexty',
                         name=f'{window_monthly} Month Rolling High'))

fig.update_xaxes(tickangle=-45, tickfont = dict(family = 'Arial', size = 14,color = 'black'))
fig.update_layout(hovermode="x unified",clickmode ='select',height=500,width=1100,)
fig.update_layout(legend=legend_config)

display(Markdown('<h2>Monthly Inflation Data</h2>'))
fig.show(config=scatter_config)

<h2>Yearly CPI Data</h2>

<h2>Monthly CPI Data</h2>

<h2>Yearly Inflation Data</h2>

<h2>Monthly Inflation Data</h2>

## Categorical Pricing data

In [4]:
cat_pricing = pd.read_excel('CategoryPriceData_BLS.xlsx',engine='openpyxl',sheet_name = 'BLS Data Series',skiprows=3)
cat_pricing = cat_pricing.loc[:,~cat_pricing.columns.str.startswith('HALF')]
cat_pricing.set_index('Series ID',inplace = True)
cat_pricing = cat_pricing.T
cat_pricing.reset_index(inplace = True)
cat_pricing.rename(columns = {'index': 'Date'},inplace = True)
cat_pricing.replace({'\n': ' '}, regex=True,inplace =True)
cat_pricing["Date"] = pd.to_datetime(cat_pricing.Date, format="%b %Y", dayfirst=True)
cat_pricing.set_index('Date',inplace = True)
display(cat_pricing.head())

series_keys = pd.read_excel('CategoryPriceData_BLS.xlsx',engine='openpyxl',sheet_name = 'Sheet1')
series_keys = series_keys[['FullCode','item_name']]
display(series_keys.head())

Series ID,CUSR0000SA0E,CUSR0000SA0L1,CUSR0000SA0L12,CUSR0000SA0L12E,CUSR0000SA0L12E4,CUSR0000SA0L1E,CUSR0000SA0L2,CUSR0000SA0L5,CUSR0000SA0LE,CUSR0000SA311,...,CUSR0000SEHE,CUSR0000SEHE01,CUSR0000SEHE02,CUSR0000SEHF,CUSR0000SEHF01,CUSR0000SEHF02,CUSR0000SEHG,CUSR0000SEHG01,CUSR0000SEHG02,CUSR0000SEHH
Date,,,,,,,,,,,,,,,,,,,,,
1947-01-01,NaN,21.0,NaN,NaN,NaN,NaN,23.2,NaN,NaN,46.1,...,8.5,NaN,NaN,18.3,NaN,NaN,NaN,NaN,NaN,NaN
1947-02-01,NaN,21.1,NaN,NaN,NaN,NaN,23.3,NaN,NaN,46.5,...,8.5,NaN,NaN,18.3,NaN,NaN,NaN,NaN,NaN,NaN
1947-03-01,NaN,21.3,NaN,NaN,NaN,NaN,23.8,NaN,NaN,47.1,...,8.5,NaN,NaN,18.3,NaN,NaN,NaN,NaN,NaN,NaN
1947-04-01,NaN,21.4,NaN,NaN,NaN,NaN,23.8,NaN,NaN,47.5,...,8.7,NaN,NaN,18.3,NaN,NaN,NaN,NaN,NaN,NaN
1947-05-01,NaN,21.5,NaN,NaN,NaN,NaN,23.7,NaN,NaN,47.6,...,8.7,NaN,NaN,18.3,NaN,NaN,NaN,NaN,NaN,NaN


,FullCode,item_name
0,CUSR0000AA0,All items - old base
1,CUSR0000AA0R,Purchasing power of the consumer dollar - old ...
2,CUSR0000SA0,All items
3,CUSR0000SA0E,Energy 1
4,CUSR0000SA0L1,All items less food


## Import item price data

### We have to use unadjusted data for this since there is no data available for seasonally adjusted item data

In [16]:
cpi_unadj = pd.read_excel('CPIRawData_unadj.xlsx',engine='openpyxl',sheet_name = 'BLS Data Series',skiprows=11)
cpi_unadj = cpi_unadj.loc[:,~cpi_unadj.columns.str.startswith('HALF')]
cpi_unadj.drop(columns = ['Annual'], inplace = True)

#use pd.melt to combine columns and rows
cpi_unadj = pd.melt(cpi_unadj, id_vars=["Year"], var_name="Month", value_name = "CPI")
#modify columns to get usable format and calculate inflation
cpi_unadj['Date'] = cpi_unadj.Month + " " + cpi_unadj['Year'].map(str)
cpi_unadj = cpi_unadj[['Date','CPI']]
cpi_unadj["Date"] = pd.to_datetime(cpi_unadj.Date, format="%b %Y", dayfirst=True)
cpi_unadj = cpi_unadj.sort_values("Date")
#cpi_monthly ['Date'] = cpi_monthly['Date'].dt.strftime('%m/%d/%Y')
cpi_unadj ['Date'] = cpi_unadj['Date'].dt.date

display(Markdown('<h2>Unadjusted CPI Data</h2>'))
display(cpi_unadj.head())

<h2>Unadjusted CPI Data</h2>

,Date,CPI
0,1913-01-01,9.8
110,1913-02-01,9.8
220,1913-03-01,9.8
330,1913-04-01,9.8
440,1913-05-01,9.7


In [5]:
pricing = pd.read_excel('IndividualPricingData_BLS.xlsx',engine='openpyxl',sheet_name = 'BLS Data Series',skiprows=3)
pricing.set_index('Series ID',inplace = True)
pricing = pricing.T
pricing.reset_index(inplace = True)
pricing.rename(columns = {'index': 'Date'},inplace = True)
pricing.replace({'\n': ' '}, regex=True,inplace =True)
pricing["Date"] = pd.to_datetime(pricing.Date, format="%b %Y", dayfirst=True)
pricing.set_index('Date',inplace = True)
display(pricing.head())

series_keys = pd.read_excel('IndividualPricingData_BLS.xlsx',engine='openpyxl',sheet_name = 'ItemCodeKeys')
series_keys = series_keys[['FullCode','item_name']]
display(series_keys.head())

Series ID,APU0000701111,APU0000701311,APU0000701312,APU0000701321,APU0000701322,APU0000702111,APU0000702112,APU0000702211,APU0000702212,APU0000702213,...,APU0000FD2101,APU0000FD3101,APU0000FD4101,APU0000FF1101,APU0000FJ1101,APU0000FJ4101,APU0000FL2101,APU0000FN1101,APU0000FN1102,APU0000FS1101
Date,,,,,,,,,,,,,,,,,,,,,
1973-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,FullCode,item_name
0,APU0000701111,"Flour, white, all purpose, per lb. (453.6 gm)"
1,APU0000701311,"Rice, white, long grain, precooked (cost per p..."
2,APU0000701312,"Rice, white, long grain, uncooked, per lb. (45..."
3,APU0000701321,Spaghetti (cost per pound/453.6 grams)
4,APU0000701322,"Spaghetti and macaroni, per lb. (453.6 gm)"


In [6]:
item_counts = {}

for column in pricing.columns:
    temp_series = pricing[column].dropna()
    item_datapoints = len(temp_series)
    item_counts[column] = item_datapoints

df_item_counts = pd.DataFrame(item_counts, index=['DataPoints']).T
df_item_counts.reset_index(inplace = True)
df_item_counts.rename(columns = {'index': 'ItemNames'},inplace = True)

fig = px.bar(df_item_counts, x='ItemNames', y='DataPoints')
fig.update_layout(height=700,width=1100,)
display(Markdown('<h2>Number of Data Points per Item</h2>'))
fig.show()


<h2>Number of Data Points per Item</h2>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c5ceaca9-e365-4132-a867-b543164c96b0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>